In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("imagefolder", data_dir="../experimental/imgs/", split='train')

Resolving data files:   0%|          | 0/275 [00:00<?, ?it/s]

Found cached dataset imagefolder (/Users/yusuke-fukasawa/.cache/huggingface/datasets/imagefolder/default-0ad062e79173203c/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


In [3]:
dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 275
})

In [4]:
split_dataset = dataset.train_test_split(test_size=0.1)

In [21]:
from transformers import SwinForImageClassification, SwinConfig, TrainingArguments, Trainer

In [7]:
from datasets import load_metric

metric = load_metric("accuracy")

/var/folders/tf/xcqcrf2n4kqgmmsn4w_t22b80000gn/T/ipykernel_22248/1780215247.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [8]:
example = split_dataset["train"][10]
example

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=70x65>,
 'label': 13}

In [9]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("fufufukakaka/autotrain-pokemon-image-classification-2-47828116910")
feature_extractor

/Users/yusuke-fukasawa/Library/Caches/pypoetry/virtualenvs/poke-battle-logger-xYV7PrtE-py3.10/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


ViTFeatureExtractor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "feature_extractor_type": "ViTFeatureExtractor",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ViTFeatureExtractor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [34]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
train_transforms = Compose(
        [
            RandomResizedCrop((feature_extractor.size["width"], feature_extractor.size["height"])),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

val_transforms = Compose(
        [
            Resize((feature_extractor.size["width"], feature_extractor.size["height"])),
            CenterCrop((feature_extractor.size["width"], feature_extractor.size["height"])),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

In [31]:
labels = split_dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[2]

'アラブルタケ'

In [35]:
train_ds = split_dataset['train']
val_ds = split_dataset['test']

train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

In [36]:
train_ds[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=70x65>,
 'label': 25,
 'pixel_values': tensor([[[1.8037, 1.8037, 1.8037,  ..., 1.8037, 1.8037, 1.8037],
          [1.8037, 1.8037, 1.8037,  ..., 1.8037, 1.8037, 1.8037],
          [1.8037, 1.8037, 1.8037,  ..., 1.8037, 1.8037, 1.8037],
          ...,
          [1.9920, 1.9920, 1.9920,  ..., 2.1633, 2.1633, 2.1633],
          [1.9920, 1.9920, 1.9920,  ..., 2.1633, 2.1633, 2.1633],
          [1.9920, 1.9920, 1.9920,  ..., 2.1633, 2.1633, 2.1633]],
 
         [[1.9734, 1.9734, 1.9734,  ..., 1.9734, 1.9734, 1.9734],
          [1.9734, 1.9734, 1.9734,  ..., 1.9734, 1.9734, 1.9734],
          [1.9734, 1.9734, 1.9734,  ..., 1.9734, 1.9734, 1.9734],
          ...,
          [2.2185, 2.2185, 2.2185,  ..., 2.3410, 2.3410, 2.3410],
          [2.2185, 2.2185, 2.2185,  ..., 2.3410, 2.3410, 2.3410],
          [2.2185, 2.2185, 2.2185,  ..., 2.3410, 2.3410, 2.3410]],
 
         [[2.2217, 2.2217, 2.2217,  ..., 2.2217, 2.2217, 2.2217],
     

In [37]:
from transformers import AutoFeatureExtractor, AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained("fufufukakaka/autotrain-pokemon-image-classification-2-47828116910",
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes = True)

Some weights of SwinForImageClassification were not initialized from the model checkpoint at fufufukakaka/autotrain-pokemon-image-classification-2-47828116910 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([93, 1024]) in the checkpoint and torch.Size([98, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([93]) in the checkpoint and torch.Size([98]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
model_name = "pokemon_image_classifier"
batch_size = 32

args = TrainingArguments(
    model_name,
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

In [18]:
import numpy as np

# the compute_metrics function takes a Named Tuple as input:
# predictions, which are the logits of the model as Numpy arrays,
# and label_ids, which are the ground-truth labels as Numpy arrays.
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [19]:
import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [39]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

In [40]:
train_results = trainer.train()
# rest is optional but nice to have
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/Users/yusuke-fukasawa/Library/Caches/pypoetry/virtualenvs/poke-battle-logger-xYV7PrtE-py3.10/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,4.421220,0.000000
2,No log,4.046308,0.107143
3,No log,3.605418,0.178571
4,No log,3.203001,0.250000
5,3.845400,2.900344,0.321429
6,3.845400,2.602329,0.464286
7,3.845400,2.262529,0.642857
8,3.845400,1.987737,0.750000
9,3.845400,1.759317,0.785714
10,1.686400,1.548900,0.821429


***** train metrics *****
  epoch                    =        20.0
  total_flos               = 360855898GF
  train_loss               =      1.7128
  train_runtime            =  0:19:34.80
  train_samples_per_second =       4.205
  train_steps_per_second   =       0.034


In [41]:
metrics = trainer.evaluate()
# some nice to haves:
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =       20.0
  eval_accuracy           =     0.9286
  eval_loss               =     0.9008
  eval_runtime            = 0:00:02.50
  eval_samples_per_second =     11.188
  eval_steps_per_second   =        0.4


In [49]:
# 推論テスト
import cv2
from PIL import Image
from transformers import pipeline

classifier_pipe = pipeline(task="image-classification", model=trainer.model, feature_extractor=feature_extractor)
pokemon_image = cv2.imread("../template_images/labeled_pokemon_templates/チヲハウハネ_1920.png")
pokemon_image2 = cv2.cvtColor(pokemon_image, cv2.COLOR_BGR2RGB)
pokemon_image3 = Image.fromarray(pokemon_image2)
classifier_pipe(pokemon_image3)

[{'score': 0.9311748743057251, 'label': 'チヲハウハネ'},
 {'score': 0.004140698350965977, 'label': 'ウルガモス'},
 {'score': 0.0037405183538794518, 'label': 'ルガルガンたそがれ'},
 {'score': 0.00328653072938323, 'label': 'ジバコイル'},
 {'score': 0.0031914354767650366, 'label': 'ラウドボーン'}]